### Prompts

In this notebook we will go over the following topics-

1. Prompt Templated
2. Connecting to Feature Store

In [1]:
from langchain import PromptTemplate

In [2]:
template = """
I want you to act as a naming consultant for new companies. What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables = ["product"],
    template = template,
)

prompt.format(product = "colorful gloves")

'\nI want you to act as a naming consultant for new companies. What is a good name for a company that makes colorful gloves?\n'

In [3]:
template = "Tell me a {adjective} joke about {content}."

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['adjective', 'content'], output_parser=None, partial_variables={}, template='Tell me a {adjective} joke about {content}.', template_format='f-string', validate_template=True)

In [4]:
prompt_template.format(adjective = "funny", content = "chickens")

'Tell me a funny joke about chickens.'

In [5]:
# Serialize the prompt template -
prompt_template.save("test_prompt.json")

In [6]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt('test_prompt.json')

In [7]:
assert prompt_template == loaded_prompt

Useful prompts from langchain : https://github.com/hwchase17/langchain-hub/tree/master/prompts

### Pass few shot examples to a prompt template

In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [9]:
# create the list of few shot examples -
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [11]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables = ["word", "antonym"],
    template = example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = "Word: {input}\nAntonym:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [12]:
print(few_shot_prompt.format(input = "little"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: little
Antonym:


### large number of examples -

In [13]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [16]:
example = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]


In [17]:
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 25,
)

In [18]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym for every input",
    suffix = "Word: {input}\nAntonym:",
    input_variables = ["input"],
    example_separator = "\n\n"
)

In [19]:
print(dynamic_prompt.format(input = "big"))

Give the antonym for every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


### Connecting to Feature Store